## Find the peaks of the extracted growth line pixels

The final stage of the process is to extract the peaks of the pixel intensities
and measure the distance between them, and, because the peaks
should correspond to the segmented rings, this is analogous to hand-measuring
the distance between rings. The peaks are found via the use of the
[`scipy.signal.find_peaks`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html)
function. It relies on the notion of
[peak prominence](https://en.wikipedia.org/wiki/Topographic_prominence) to
determine which peaks in the signal, i.e., the vector of pixel intensities, are
really *peaks* in the signal and which are just noise. Full details can be
found in the project [README](./README.md).

In [ ]:
import os
import yaml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import find_peaks
from shellai import peakfinding

# load the config file
with open("config.yaml", "r") as fd:
    cfg = yaml.safe_load(fd)

In [ ]:
# # #  General settings
# base directory where we'll be saving things to
dir_base = cfg['paths']['local']['base']

# directory where the predictions are stored
dir_predictions = "output"

# directory where the ring widths and plots will be saved
dir_saved_widths = "ring_predictions"

# settings for both sets of predictions
experiment_info = {
    "loo": {
        "image_names": cfg['images']['train_images'],
        "file_mask": "loo_{image_name:s}_all_predictions.npz",
        "base_image_dir": cfg['images']['train_dir']
    },

    "final": {
        "image_names": cfg['images']['test_images'],
        "file_mask": "final_{image_name:s}_all_predictions.npz",
        "base_image_dir": cfg['images']['test_dir']
    }
}

# location to a spreadsheet which contains the scale of the images
# specifically, it contains the length in micrometers (μm) and its
# corresponding length in pixels. see the included file for its structure:
# pixel_absolute_conversions.xlsx
conversion_spreadsheet_location = os.path.join(
    dir_base,
    "pixel_absolute_conversions.xlsx",
)

# whether to also plot the results
create_plots = True

# # # Peak finding settings
# list of epochs we wish to get the peaks of the extracted growth line from
epochs = [100, 150, 200]

# for full details of how the peak finding works, and what the following
# parameters do, please see:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html

# list of prominences to extract for
peak_prominences = [0.05, 0.1]

# minimum width of peaks
peak_width = 1

# minimum distance between peaks (in pixels). this value was selected by
# looking at the minimum distance between the peaks training images (~30) and
# choosing a smaller value (25)
peak_min_dist = 25

In [ ]:
# first, we extract the scaling from pixel length to micrometer
# this uses the first row of the spreadsheet as column names in the dataframe
df = pd.read_excel(conversion_spreadsheet_location)

pix_to_um = {}

# iterate through each row of the dataframe
for _, row in df.iterrows():

    # get the image name
    image_name = str(row["Source"])

    # extract the value by which to multiply pixel distances to convert to um
    scale = row["Length (um)"] / row["Length (pixels)"] # type:ignore
    pix_to_um[image_name] = scale

In [ ]:
# gather all the experiments to extract the rings for
exps = [(e, p) for e in epochs for p in peak_prominences]
n_exps = len(exps)

for experiment_name in experiment_info:
    image_names = experiment_info[experiment_name]["image_names"]
    file_mask = experiment_info[experiment_name]["file_mask"]
    base_image_dir = experiment_info[experiment_name]["base_image_dir"]

    for image_name in image_names:
        pred_path = os.path.join(
            dir_base, 
            dir_predictions,
            file_mask.format(image_name=image_name)
        )

        # get the measurements along the growth line (ring_data_ and their
        # corresponding pixel coordinates (flc)
        with np.load(pred_path, allow_pickle=True) as fd:
            # dictionary keys are ['preds', 'gt']
            ring_data = fd["ring_data"].item()

            # coordinates of the line along which the ring data was extracted
            # this should be shape (n, 2)
            flc = fd["flc"]

        # scaling to convert px to um
        px_to_um_scaling = pix_to_um[image_name]

        base_image_dir = experiment_info[experiment_name]["base_image_dir"]
        image_dir = os.path.join(base_image_dir, image_name)

        # ground-truth distances in um. these are hand-measured
        gt_ring_distances_um = peakfinding.parse_measurement_csv_from_directory(
            image_dir
        )

        # storage for the extracted ring widths and ground truth
        results = {}
        results['original'] = gt_ring_distances_um

        # for combination of epoch and prominence value
        for (epoch, prominence) in exps:
            # get the pixel values along the axis
            r = ring_data['preds'][epoch]

            # normalise them such that the largest value is one and the
            # smallest is zero. note that this is done so that the prominence
            # value, has as close to the same meaning for each set of pixels
            rnormed = peakfinding.normalize_vector(r)

            # find the peaks
            r_peaks, _ = find_peaks(
                rnormed, 
                prominence=prominence, 
                width=peak_width,
                distance=peak_min_dist,
            )

            # calculate the distances between the extracted peaks,
            # rescale them, and reverse the order. the latter is carried out
            # to match the ground truth
            predicted_ring_distances_um = peakfinding.get_predicted_ring_distances(
                flc, r_peaks, px_to_um_scaling, reversed=True
            )
            
            # store the resulting vector of ring widths
            results[(epoch, prominence)] = predicted_ring_distances_um

        # create a pandas dataframe containing the ring widths by starting
        # with the ground truth in one column
        df = pd.DataFrame(results['original'], columns=['Ground Truth'])

        # and adding columns corresponding to each prominence and epoch combo
        for key in exps:
            df = pd.concat([df, pd.DataFrame(results[key], columns=[key])], axis=1)

        # next, rename the index appropriately
        df = df.rename_axis('Ring Number')

        # and finally, save the csv file
        csv_file_path = os.path.join(
            dir_base, 
            dir_predictions, 
            dir_saved_widths,
            f"{experiment_name}_{image_name}_rings.csv", 
        )
        df.to_csv(csv_file_path, float_format="%0.2f")

        print(f'Saved: {csv_file_path}')

        # if set to True, we create a plot of each experiment for a given image
        if create_plots:
            image_file_path = os.path.join(
                dir_base, 
                dir_predictions, 
                dir_saved_widths,
                f"{experiment_name}_{image_name}_rings.pdf", 
            )

            gt = results['original']
            gtmax = np.max(results['original'])
            maxlen = np.max([len(x) for x in results.values()])

            fig, axes = plt.subplots(
                n_exps, 1, figsize=(15, 3 * n_exps), sharex=True, dpi=200
            )
            for i, (epoch, prominence) in enumerate(exps):
                ax = axes[i]
                lbl = f"Epochs={epoch}, Prominence={prominence}"
                pred = results[(epoch, prominence)]

                gtx = np.arange(maxlen - gt.size, maxlen)
                gtp = np.arange(maxlen - pred.size, maxlen)

                ax.plot(gtx, gt, 'o-', label='Ground truth')
                ax.plot(gtp, pred, 'x-', label=lbl)
                ax.legend(loc="upper right")

                ax.set_ylim([0, gtmax+ 50])

            plt.savefig(image_file_path, bbox_inches='tight')
            plt.close()

            print(f'Saved: {image_file_path}')